In [1]:
# 分割出嘴唇区域，中心对齐的方法
'''
分离出嘴部的区域，使用alignment信息
'''
import numpy as np
import face_alignment
import cv2
import os
from skimage import io
from matplotlib import pyplot as plt
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
%matplotlib inline
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)

In [6]:
norm_width =  240
norm_height = 120
span_width = int(norm_width/2)
span_height = int(norm_height/2)
landmark_path = '/home/fesian/contest_workspace/chunyu/dataset/landmarks_normal/'
images_path = '/home/fesian/contest_workspace/chunyu/dataset/lip_rawdata/'
save_image_path = '/home/fesian/contest_workspace/chunyu/dataset/'
tmode = ('test', '')
mouth_bias_x = 20
mouth_bias_y = 20
invalid_list = []

In [7]:
import time
for dataset in tmode:
    lpath = landmark_path+'{}data/'.format(dataset)
    ipath = images_path+'lip_{}/lip_{}/'.format(dataset,dataset)
    dirlist = os.listdir(lpath)
    for sample in dirlist:
        begintime = time.clock()
        spath = lpath+sample
        landmark = np.load(spath)
        if landmark.shape[0] == 0: 
            invalid_cell = {
                'name': sample,
                'data_type': dataset,
                'err_type': 'landmark_zero_invalid'
            }
            invalid_list.append(invalid_cell)
            continue
        img_path = ipath+'{}/'.format(sample[:-4])
        imgslist = os.listdir(img_path)
        for ind, imgsample in enumerate(imgslist):
            imgpath = img_path+imgsample
            imgmat = io.imread(imgpath)
            align = fa.get_landmarks(imgmat)
            if not align: 
                invalid_cell = {
                    'name': sample,
                    'data_type': dataset,
                    'err_type': 'landmark_extraction_fail'
                }
                invalid_list.append(invalid_cell)
                continue
            align = align[0][48:68,:]
            max_x, max_y, min_x, min_y = np.max(align[:,0]), np.max(align[:,1]), np.min(align[:,0]), np.min(align[:,1])
            height, width, depth = imgmat.shape
            center_width, center_height = int((max_x+min_x)/2), int((max_y+min_y)/2)
            width_cut, height_cut = max_x-min_x, max_y-min_y
            
            if width_cut > norm_width or height_cut > norm_height:
                # 标准化区域小于有效区域，以有效区域切割，并且进行尺度放缩归一化
                cuty = int(min_y)-mouth_bias_y if int(min_y)-mouth_bias_y > 0 else 0
                cutx = int(min_x)-mouth_bias_x if int(min_x)-mouth_bias_x > 0 else 0
                mouthimg = imgmat[cuty:int(max_y)+mouth_bias_y, cutx:int(max_x)+mouth_bias_x, :]
                try:
                    mouthimg = cv2.resize(mouthimg, (norm_width, norm_height))
                except:
                    invalid_cell = {
                        'name': sample,
                        'data_type': dataset,
                        'err_type': 'resize_fail'
                    }
                    invalid_list.append(invalid_cell)
                    continue
            else:
                cutheight = center_height-span_height if center_height-span_height > 0 else 0
                cutwidth = center_width-span_width if center_width-span_width > 0 else 0
                mouthimg = imgmat[cutheight:center_height+span_height, cutwidth:center_width+span_width, :]
            
            if not os.path.exists(save_image_path+'center_mouth_rgb/{}/{}/'.format(dataset,sample[:-4])):
                os.makedirs(save_image_path+'center_mouth_rgb/{}/{}/'.format(dataset,sample[:-4]))
            save_path = save_image_path+'center_mouth_rgb/{}/{}/{}.jpg'.format(dataset,sample[:-4],imgsample[:-4])
            cv2.imwrite(save_path, mouthimg[:,:,::-1])
        endtime = time.clock()
        print('cost time = {}, at file {}'.format(endtime-begintime, sample))
    # write the log
    invalid_list = np.array(invalid_list)
    log_file_path = '/home/fesian/contest_workspace/chunyu/dataset/logs/'
    np.save(log_file_path+'{}_invalid_mouth_extraction.log'.format(dataset), invalid_list)

cost time = 0.18839699999989534, at file af1b56a5b4695665a6e70ca8131a3daa.npy
cost time = 0.3513110000001234, at file 6ddeb70fc4382bfc69911376501595de.npy
cost time = 0.27942799999982526, at file d2bb47b4294981a12f6a604bd826b503.npy
cost time = 0.3052860000000237, at file 08412ef56dcce4db771336b6c19516a7.npy
cost time = 0.4401100000000042, at file 97c8695f62841d6106392dc5d9b01571.npy
cost time = 0.2480070000001433, at file 6be1a65e65fc7ba639f865c2525549e1.npy
cost time = 0.564792000000125, at file 1dafa95734a3e35117197c814f25636c.npy
cost time = 0.7103060000001733, at file c593106b85d9a57f57f25e8d611cb059.npy
cost time = 0.31752800000003845, at file 13a804655c2003272bd12a2246f50b14.npy
cost time = 0.27497699999980796, at file 841225bd477eb1ad5c7f51b89d932905.npy
cost time = 0.4438249999998334, at file 954db007136491ce578810c9b398efbc.npy
cost time = 0.24369099999989885, at file 810713aad27b9f73373c2b081544d6c7.npy
cost time = 0.2821279999998296, at file 8db0626643b0f0126407bca718c5151c

cost time = 0.24433399999998073, at file 4b8d751cd2543eddaccd57bea548287b.npy
cost time = 0.12962599999991653, at file bff718566f51c65de5625a6d7b9b5e7e.npy
cost time = 0.2246820000000298, at file a7b983af50944dbbfbdb893f3e044608.npy
cost time = 0.2680490000000191, at file 50d8b7485b490ace6ec350ee8c2a1b90.npy
cost time = 0.24639799999999923, at file b1073eb0fb5cd9e44b6343d7f426a347.npy
cost time = 0.24636600000008002, at file d458fd08072964538cd217aa513d5e92.npy
cost time = 0.1771009999999933, at file 694484581fd1ee040441eb6748e59891.npy
cost time = 0.2358390000001691, at file 6ce1fb9cb1a2ccdca4e1dd94dcea7e0c.npy
cost time = 0.3429719999999179, at file a400b8cc5a8fb505e8d27cc134fb4eaf.npy
cost time = 0.4490570000000389, at file 403ec43516403d9ee1f9ebf327bc0d71.npy
cost time = 0.14790300000004208, at file cfd42d1383d0423aab3caabe38d9ebcd.npy
cost time = 0.5809650000001056, at file a665e9398a35ce80147ea82782622704.npy
cost time = 0.2447140000001582, at file f3b3bc3b17ea7e584f332a277356055

cost time = 0.37418300000013005, at file f6242e61a38acf6361a4d2215259fb54.npy
cost time = 0.178717000000006, at file e018e40f53044c9789510d7681e98677.npy
cost time = 0.38177999999993517, at file 0ce7c86420ba52153417c436085f93e0.npy
cost time = 0.1930440000000999, at file ca7d21411925fa8e18357d673fd458a4.npy
cost time = 0.2800879999999779, at file 1a17673e482d0ef0c2ccd4dfdc02f53f.npy
cost time = 0.2856939999999213, at file a585d20ca035bfdea4ba3d2b837cc29f.npy
cost time = 0.3578410000000076, at file 2bd3c39af796a1adbdb75a3d5e547643.npy
cost time = 0.35080999999991036, at file 9300385b96740d000199930b18b3f20e.npy
cost time = 0.2891300000001138, at file cb73f4058b5216806569f6e35155a798.npy
cost time = 0.2800680000000284, at file efbad72d8951cfc9538c97eb50137b52.npy
cost time = 0.6115780000000086, at file 8f241aa91598a91070555873dd0f3078.npy
cost time = 0.3828329999998914, at file 05273d7114f6e07f8fb3d1c09ee1c162.npy
cost time = 0.2894509999998718, at file 167e08f219999b496b2405b69816a877.n

cost time = 0.3740619999998671, at file 5174e732ba514d8213276664452171ff.npy
cost time = 0.34876900000017486, at file 8ad37e883673692c7fd2b4222c270f18.npy
cost time = 0.37451899999996385, at file c2d441bdc21837c0320f61fc58456e2b.npy
cost time = 0.2999919999999747, at file affea48c967fec13c8623f10e1140f4b.npy
cost time = 0.7780999999999949, at file 0ae2c7652b6465a87a379bd933345488.npy
cost time = 0.3039610000000721, at file 6554014ce4c7c8cd2bd0f577081af837.npy
cost time = 0.6720410000000356, at file 4d026048c607d40eb0ff2b173e36e754.npy
cost time = 0.6645989999999529, at file f3302a89d703b00186dd590eba6b7a44.npy
cost time = 0.4277209999997922, at file 9f9df820b0f42ff5ab1239ab59391bc8.npy
cost time = 0.3997659999999996, at file e235efdb288568b1e2935d6669f6d867.npy
cost time = 0.5878920000000107, at file 6facb5cd53eb1b1ed473752afcb64eac.npy
cost time = 0.2684389999999439, at file 97dd81137e3ca16535d7e3b59fc78f0a.npy
cost time = 0.32295300000009775, at file 8442ac06feba090734973b983a4d3de7.

cost time = 0.46727400000008856, at file 78c2aa7f8f2d51880731b6d6f9a3eaf7.npy
cost time = 0.8566359999999804, at file 4229057008c0020b66a97e0eb1fba248.npy
cost time = 0.3284320000000207, at file 4051bea1508c7f7a4b2d4489cafea994.npy
cost time = 0.31241300000010597, at file 964c994f3f9e657fd6f27fabd83f4e09.npy
cost time = 0.6629479999999148, at file 120da396299850a2072fdd57422489bd.npy
cost time = 0.5591830000000755, at file 0ef76fa3412ac0e65175e1110fd36a07.npy
cost time = 0.2780170000000908, at file 1079b829fe410ed8c92c981f080d453d.npy
cost time = 0.29537299999992683, at file 40db3c482b528e359e8f49990f7d2066.npy
cost time = 0.4984230000000025, at file 39c4496c3d3c689ad80eb90eb7986b11.npy
cost time = 0.2381710000001931, at file 0da0bc55ea1d8c3e39b6dfcbbb0ab8a7.npy
cost time = 0.476374000000078, at file 2d17f0f9fa7143c59fe1be4388258db6.npy
cost time = 0.17728899999997338, at file 939b6635513e2d10118d9989f361be00.npy
cost time = 0.24531600000000253, at file a260f6437764c6e4bde602afc803c29b

...

cost time = 0.3374549999998635, at file 4fee16447c4808dbce725add11f9fd1d.npy
cost time = 0.2593050000000403, at file 2b9d9a1655396d79fa9d102c311354b7.npy
cost time = 0.4091640000001462, at file b229893f6edc51a061aab815f7d6e829.npy
cost time = 0.29144399999995585, at file e1957ee61200aea456dbf2f48c2350a2.npy
cost time = 0.5531860000000961, at file 068413fa14818211e1c0983f126e50a7.npy
cost time = 0.21980899999994108, at file d715d98907e91d830565eecfcca1d8ca.npy
cost time = 0.7616729999999734, at file 7283e01dc6bd52626859028766e7fc61.npy
cost time = 0.571536000000151, at file 59784900c4f229dcaeb6a974db8c872c.npy
cost time = 0.24663799999984803, at file 8ee2be2e3070552b04dc8a8ad0ec45e8.npy
cost time = 0.4361159999998563, at file 3ba8e99a26c68ae1344c887b9323b2ae.npy
cost time = 0.2491229999998268, at file 5b20731e622e960d3031dfe65af015e8.npy
cost time = 0.45897500000000946, at file d2a212030fef5bc084a5a0c691cee3a4.npy
cost time = 0.4937670000001617, at file 84fa375d4af1b1758c323f8a0c83dc02.

cost time = 0.2485359999998309, at file ec2935c475fbbf96800a9ce1d38bc7fc.npy
cost time = 0.2493170000000191, at file f314e733723cf3541d6e21bd3f0dd1f8.npy
cost time = 0.4586309999999685, at file 2a194aac21bdd85627919149905b4aa9.npy
cost time = 0.3842810000001009, at file 7918a788805d8df564a6493375329d4b.npy
cost time = 0.46635700000001634, at file aa8e3c45d39fe3d15094f3e5ad93b534.npy
cost time = 0.4024979999999232, at file fe86d5c89626ffb715dab97eb129c274.npy
cost time = 0.2848180000000866, at file edbab780c4bc709013ee92143e15ef9d.npy
cost time = 0.8122020000000703, at file a0173591241c88bb1dc1bf25c3962c3d.npy
cost time = 0.4055039999998371, at file cbc24e724246e2cedc8ffaae662ad418.npy
cost time = 1.0532889999999497, at file ee2aaefc67402d5119db617b451afa78.npy
cost time = 0.26329499999997097, at file 5b2dcc4268b1cabc3c9434a33e7a141a.npy
cost time = 0.712887000000137, at file 7347a51fbb283c0b8c8878a2ef682baf.npy
cost time = 0.26799700000015036, at file 87d11b050fe957633fcfc6806c97a7f2.n

cost time = 0.5209109999998418, at file 9efadc9a20136cea388395152a617d86.npy
cost time = 0.1836350000000948, at file 3dda1fb87174960baa78efd58e1eb25a.npy
cost time = 0.29905200000007426, at file 96bcdc7605bd3c394c3abee8c6f9b573.npy
cost time = 0.5585320000000138, at file fe557c7507b597e994aa4519f9a60cf2.npy
cost time = 0.2727749999999105, at file 720096eff10a9acb720beec6c9b862ba.npy
cost time = 0.33300499999995736, at file 2f27b6c0129178616582aee53d5440d3.npy
cost time = 0.7123469999999088, at file 77261598e133cfcfac8d87d736ee49c8.npy
cost time = 0.2503790000000663, at file 5702182d5c14962d273808c4ad4d9c9b.npy
cost time = 0.20623699999987366, at file d6ccfffdfd1b5c7f2e76878777b88c65.npy
cost time = 0.48150900000018737, at file e7d62352acfcf383a09d134be947eb61.npy
cost time = 0.41208800000003976, at file f3d6ddd5b9c68c80deca9aa64aaee265.npy
cost time = 0.24707699999999022, at file fa956e4ff7c745fa78701cad082d6541.npy
cost time = 0.4939110000000255, at file d3caf31ec54d107f8e68ce4e4ffc48

cost time = 0.4463050000001658, at file 78b05d078cd0820c136a004b79f86c4d.npy
cost time = 0.2970990000001166, at file e0eae9b5d62a10d330ae222ea963deeb.npy
cost time = 0.758591000000024, at file ec0008c52e6319b954877d2896bd5003.npy
cost time = 0.26893199999994977, at file 38752f9b6bf0c4df7e486e6fb464d507.npy
cost time = 0.3347449999998844, at file f27a0fc41df9bf44b8c5383305b27b0b.npy
cost time = 0.5051109999999426, at file e9cf7062fedc2cb615bf63fb18529aec.npy
cost time = 0.2546439999998711, at file 3d16e32401f6307db87f86ee1e17ad3c.npy
cost time = 0.35172399999987647, at file 4bd3d3cb4a068cda47a416bedb7218b1.npy
cost time = 0.19501199999990604, at file 2fce52efb09c53cdafc3a7ea45d0e734.npy
cost time = 0.2981079999999565, at file 177a503e25216a8a0092bd040ab37ec0.npy
cost time = 0.48018900000010944, at file 6b9a5ea3778c7888e4fe986c55e0e2cf.npy
cost time = 0.4198069999999916, at file bc386b4b9946fdd31fc14870a6524abc.npy
cost time = 0.3750039999999899, at file a89e79a421130bfcdbe8f0144aedd67e.

TypeError: save() missing 1 required positional argument: 'arr'

In [8]:
np.save(log_file_path+'{}_invalid_mouth_extraction.log'.format(dataset), invalid_list)